In [0]:
!pip install dash==1.12.0

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [0]:
!unzip ngrok-stable-linux-amd64.zip

In [0]:
%%writefile app1.py

# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
    

In [0]:
get_ipython().system_raw('./ngrok http 8050 &')

In [0]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!python app1.py

In [0]:
!pip install dash-bio==0.4.8

In [0]:
%%writefile app2.py

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bio as dashbio
import pandas as pd

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files' +
    '/master/clustergram_mtcars.tsv',
    sep='	', skiprows=4
).set_index('model')

data = df.values

component = dcc.Graph(figure=dashbio.Clustergram(
  data=data,
  color_threshold={'row': 150,
  'col': 700},
  column_labels=list(df.columns.values),
  row_labels=list(df.index),
  hidden_labels=['row'],
  width=600
))

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(component)
if __name__ == '__main__':
    app.run_server(debug=True)


In [0]:
import pandas as pd

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files' +
    '/master/clustergram_mtcars.tsv',
    sep='	', skiprows=4
).set_index('model')

data = df.values

In [0]:
df

In [0]:
data

In [0]:
get_ipython().system_raw('./ngrok http 8050 &')

In [0]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!python app2.py

In [0]:
%%writefile app3.py

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bio as dashbio
import pandas as pd

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files' +
    '/master/volcano_data1.csv'
)
component = dcc.Graph(figure=dashbio.VolcanoPlot(
  dataframe=df
))

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(component)
if __name__ == '__main__':
  app.run_server(debug=True)


In [0]:
get_ipython().system_raw('./ngrok http 8050 &')

In [0]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!python app3.py

In [0]:
import pandas as pd

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files' +
    '/master/volcano_data1.csv'
)

In [0]:
df

In [0]:
!mkdir dash_hannari

In [0]:
!touch dash_hannari/__init__.py

In [0]:
!pip install dash-cytoscape

In [0]:
%%writefile dash_hannari/__init__.py

import math
import dash_cytoscape as cyto

def PhylogenyTree(tree):

  nodes, edges = generate_elements(tree)
  elements = nodes + edges

  layout = {'name': 'preset'}

  stylesheet = [
      {
          'selector': '.nonterminal',
          'style': {
              'label': 'data(confidence)',
              'background-opacity': 0,
              "text-halign": "left",
              "text-valign": "top",
          }
      },
      {
          'selector': '.support',
          'style': {'background-opacity': 0}
      },
      {
          'selector': 'edge',
          'style': {
              "source-endpoint": "inside-to-node",
              "target-endpoint": "inside-to-node",
          }
      },
      {
          'selector': '.terminal',
          'style': {
              'label': 'data(name)',
              'width': 10,
              'height': 10,
              "text-valign": "center",
              "text-halign": "right",
              'background-color': '#222222'
          }
      }
  ]

  return cyto.Cytoscape(
        id='cytoscape',
        elements=elements,
        stylesheet=stylesheet,
        layout=layout,
        style={
            'height': '95vh',
            'width': '100%'
        }
    )


def generate_elements(tree, xlen=30, ylen=30, grabbable=False):
    def get_col_positions(tree, column_width=80):
        """Create a mapping of each clade to its column position."""
        taxa = tree.get_terminals()

        # Some constants for the drawing calculations
        max_label_width = max(len(str(taxon)) for taxon in taxa)
        drawing_width = column_width - max_label_width - 1

        depths = tree.depths()
        # If there are no branch lengths, assume unit branch lengths
        if not max(depths.values()):
            depths = tree.depths(unit_branch_lengths=True)
            # Potential drawing overflow due to rounding -- 1 char per tree layer
        fudge_margin = int(math.ceil(math.log(len(taxa), 2)))
        cols_per_branch_unit = ((drawing_width - fudge_margin) /
                                float(max(depths.values())))
        return dict((clade, int(blen * cols_per_branch_unit + 1.0))
                    for clade, blen in depths.items())

    def get_row_positions(tree):
        taxa = tree.get_terminals()
        positions = dict((taxon, 2 * idx) for idx, taxon in enumerate(taxa))

        def calc_row(clade):
            for subclade in clade:
                if subclade not in positions:
                    calc_row(subclade)
            positions[clade] = ((positions[clade.clades[0]] +
                                 positions[clade.clades[-1]]) // 2)

        calc_row(tree.root)
        return positions

    def add_to_elements(clade, clade_id):
        children = clade.clades

        pos_x = col_positions[clade] * xlen
        pos_y = row_positions[clade] * ylen

        cy_source = {
            "data": {"id": clade_id},
            'position': {'x': pos_x, 'y': pos_y},
            'classes': 'nonterminal',
            'grabbable': grabbable
        }
        nodes.append(cy_source)

        if clade.is_terminal():
            cy_source['data']['name'] = clade.name
            cy_source['classes'] = 'terminal'

        for n, child in enumerate(children):
            # The "support" node is on the same column as the parent clade,
            # and on the same row as the child clade. It is used to create the
            # 90 degree angle between the parent and the children.
            # Edge config: parent -> support -> child

            support_id = clade_id + 's' + str(n)
            child_id = clade_id + 'c' + str(n)
            pos_y_child = row_positions[child] * ylen

            cy_support_node = {
                'data': {'id': support_id},
                'position': {'x': pos_x, 'y': pos_y_child},
                'grabbable': grabbable,
                'classes': 'support'
            }

            cy_support_edge = {
                'data': {
                    'source': clade_id,
                    'target': support_id,
                    'sourceCladeId': clade_id
                },
            }

            cy_edge = {
                'data': {
                    'source': support_id,
                    'target': child_id,
                    'length': clade.branch_length,
                    'sourceCladeId': clade_id
                },
            }

            if clade.confidence and clade.confidence.value:
                cy_source['data']['confidence'] = clade.confidence.value

            nodes.append(cy_support_node)
            edges.extend([cy_support_edge, cy_edge])

            add_to_elements(child, child_id)

    col_positions = get_col_positions(tree)
    row_positions = get_row_positions(tree)

    nodes = []
    edges = []

    add_to_elements(tree.clade, 'r')

    return nodes, edges


In [0]:
import dash_hannari as dashhannari

In [0]:
!pip install biopython

In [0]:
!mkdir data

In [0]:
!wget https://raw.githubusercontent.com/plotly/dash-cytoscape/master/demos/data/apaf.xml

In [0]:
!mv apaf.xml data/

In [0]:
%%writefile app4.py

import dash_hannari as dashhannari
import dash
import dash_html_components as html
from Bio import Phylo
 
tree = Phylo.read('data/apaf.xml', 'phyloxml')
component = dashhannari.PhylogenyTree(tree)
 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(component)
if __name__ == '__main__':
  app.run_server(debug=True)

In [0]:
get_ipython().system_raw('./ngrok http 8050 &')

In [0]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
!python app4.py